In [3]:
from models import DecisionTransformer
from environments import Map

In [4]:
NR_ROBOTS = 5
MAP_FILE_PATH = './maps/base.txt'

map = Map(NR_ROBOTS).load(MAP_FILE_PATH)
model = DecisionTransformer(
    context_window=150,
    hidden_dim=36,
    n_heads=4,
    n_decoders=6,
    reward_dim=1,
    map_dim=map.get_area(),
    path_dim=4 * map.get_nr_robots(),
    action_dim=map.get_nr_robots(),
)

map, model

(<environments.simulation.Map at 0x765f50165a10>,
 DecisionTransformer(
   (preprocess): Preprocess(
     (reward_embed): Linear(in_features=1, out_features=36, bias=True)
     (map_embed): Linear(in_features=300, out_features=36, bias=True)
     (path_embed): Linear(in_features=80, out_features=36, bias=True)
     (action_embed): Linear(in_features=5, out_features=36, bias=True)
     (timestep_embed): Embedding(150, 36)
   )
   (decoders): Sequential(
     (0): Block(
       (attention_layer): Attention()
       (norm_mid_layer): LayerNorm((21600,), eps=21600, elementwise_affine=True)
       (forward_layer): Linear(in_features=21600, out_features=21600, bias=True)
       (norm_end_layer): LayerNorm((21600,), eps=21600, elementwise_affine=True)
     )
     (1): Block(
       (attention_layer): Attention()
       (norm_mid_layer): LayerNorm((21600,), eps=21600, elementwise_affine=True)
       (forward_layer): Linear(in_features=21600, out_features=21600, bias=True)
       (norm_end_laye

In [5]:
map.get_paths()

array([[ 1,  2,  6,  6],
       [ 3, 21,  0, 25],
       [ 4, 11,  2, 27],
       [ 1, 29,  3,  2],
       [ 0,  9,  3, 22]])